<a href="https://colab.research.google.com/github/miraqulass/MachineLearning/blob/MLPractices/PlayGenerator(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import keras
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

In [47]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
# Loading Your Own Data

from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [48]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))
# Length of text is the number of characters in it

Length of text: 1115394 characters


In [49]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [50]:
# Encoding

vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [51]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [52]:
# int to text

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [53]:
# Creating training examples

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create a stream of characters
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [54]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [55]:
def split_input_target(chunk):    # for instance: hello
  input_text = chunk[:-1]         # hell
  target_text = chunk[1:]         # ello
  return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)   # Map is used to apply the function to every entry

In [56]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [57]:
# Training batches

BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [58]:
# Building the Model

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    # Explicitly define the input with batch size
    inputs = tf.keras.Input(batch_shape=[batch_size, None])

    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    x = tf.keras.layers.LSTM(rnn_units, return_sequences=True,
                             stateful=True, recurrent_initializer='glorot_uniform')(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)

    model = tf.keras.Model(inputs, outputs)
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (64, None)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ (64, None, 256)             │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (64, None, 1024)            │       5,246,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (64, None, 65)              │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
# Creating Loss Function

for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [60]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-6.81768684e-03  3.16746044e-03  6.51276810e-03 ... -1.61180273e-03
    1.47096452e-03 -1.09821726e-02]
  [-1.26440972e-02  1.61846646e-03  4.11273818e-03 ... -5.34773455e-04
    4.79450682e-03 -7.58870831e-03]
  [-5.91232069e-03  2.54046987e-03  2.46689003e-03 ...  3.76792857e-03
    5.39564481e-03 -1.60078751e-03]
  ...
  [-6.74692681e-03 -2.91461917e-03  2.68521253e-03 ...  2.43679155e-03
    5.75147010e-03 -1.20902318e-03]
  [-8.39396100e-03 -1.11912389e-03  2.80345650e-03 ...  2.42646289e-04
    5.08643268e-03 -4.32179950e-04]
  [-2.57436023e-03 -1.17541861e-03  3.56630911e-03 ...  3.72025557e-03
    4.41325316e-03  1.66571571e-03]]

 [[-4.64778673e-03 -1.85159384e-03 -5.96689153e-03 ...  3.00986489e-04
    8.71113152e-04 -4.91641404e-04]
  [-1.03484262e-02  1.11984985e-03  1.23523560e-03 ... -2.43693776e-03
    2.25498783e-03 -1.10321697e-02]
  [-2.40328722e-03  1.22564426e-03  1.87381811e-03 ... -6.95633562e-03
    3.46291927e-03 -4.37954301e-03]
  ...
  [-5.364

In [61]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-0.00681769  0.00316746  0.00651277 ... -0.0016118   0.00147096
  -0.01098217]
 [-0.0126441   0.00161847  0.00411274 ... -0.00053477  0.00479451
  -0.00758871]
 [-0.00591232  0.00254047  0.00246689 ...  0.00376793  0.00539564
  -0.00160079]
 ...
 [-0.00674693 -0.00291462  0.00268521 ...  0.00243679  0.00575147
  -0.00120902]
 [-0.00839396 -0.00111912  0.00280346 ...  0.00024265  0.00508643
  -0.00043218]
 [-0.00257436 -0.00117542  0.00356631 ...  0.00372026  0.00441325
   0.00166572]], shape=(100, 65), dtype=float32)


In [62]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-6.8176868e-03  3.1674604e-03  6.5127681e-03 -3.6410191e-03
  5.6521804e-03  3.5542725e-03 -6.2074801e-03 -3.4959295e-03
 -6.9853704e-04  7.5092289e-04  1.4842573e-03 -1.8161153e-03
 -2.7611330e-03  3.4591747e-03 -5.3803385e-03 -3.0521877e-04
  4.0847231e-03  3.9942525e-03 -7.3202704e-03 -2.9947904e-03
 -4.3837358e-03 -3.3912002e-04  7.1774237e-04 -1.6136977e-03
  3.8199709e-04 -3.9093881e-03 -8.0855330e-04  4.5203473e-05
  4.8964550e-03  4.0648198e-03  3.0484868e-03  3.0218980e-03
  1.5649277e-03 -1.0886979e-02  7.4030616e-04 -4.6934355e-03
 -3.6673627e-03 -2.3914760e-03 -1.6895571e-03 -6.4284317e-03
 -1.6955824e-03 -1.5276377e-04  4.9723517e-03  8.1101584e-04
 -1.1577413e-03 -6.6341939e-03 -4.5550819e-03  3.4717761e-04
  1.7755768e-04 -7.2192482e-04 -7.1078038e-04 -9.2993543e-04
 -1.2899957e-04  2.1593652e-03  3.2050940e-03  1.8238241e-03
  8.0845912e-04 -3.0543569e-03  3.3014915e-03  7.0989964e-04
  6.6054903e-04  1.3242703e-03 -1.6118027e-03  1.4709645e-03
 -1.098217

In [63]:
# Determining Predicted Characters

sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"W,mqypDPbNWc'u?ypcPOe$W 3cu'DARz&zIsQ?3rHxkKF PUpTqJs.k,bs;vSabnglDf.fci\njxNJmxQcgHKyg'\niBVQgYbxIY-T"

In [64]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [65]:
model.compile(optimizer="adam", loss=loss)

In [66]:
# Creating Checkpoints

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [25]:
# Model Training

history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step - loss: 2.8687
Epoch 2/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 81ms/step - loss: 1.8477
Epoch 3/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - loss: 1.5958
Epoch 4/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - loss: 1.4718
Epoch 5/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 79ms/step - loss: 1.3997
Epoch 6/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - loss: 1.3453
Epoch 7/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 81ms/step - loss: 1.3037
Epoch 8/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 79ms/step - loss: 1.2661
Epoch 9/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 80ms/step - loss: 1.2315
Epoch 10/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - loss: 1.1997
Epoch 11/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 80ms/step - loss: 1.1657
Epoch 12/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 79ms/step - loss: 1.1341
Epoch 13/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 81ms/step - loss: 1.0999
Epoch 14/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 79ms/step - loss: 1.0657
Epoch 15/40
172/172 ━━━━━━━━━

In [67]:
# Loading Model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [68]:
# Load the latest checkpoint if it exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

if latest_checkpoint:
    print(f"Loading weights from checkpoint: {latest_checkpoint}")
    model.load_weights(latest_checkpoint)
else:
    print("No checkpoint found. Proceeding without loading weights.")

# Ensure the model is built before loading weights
model.build(tf.TensorShape([1, None]))


No checkpoint found. Proceeding without loading weights.


In [69]:
checkpoint_num = 10
checkpoint_path = f'./training_checkpoints/ckpt_{checkpoint_num}.weights.h5'

model.build(tf.TensorShape([1, None]))

try:
    model.load_weights(checkpoint_path)
    print(f"Weights from checkpoint {checkpoint_num} loaded successfully.")
except Exception as e:
    print(f"Error loading weights from checkpoint {checkpoint_num}: {e}")

Weights from checkpoint 10 loaded successfully.


In [77]:
# Generating Text

def generate_text(model, start_string):

  num_generate = 800    # Number of characters to generate

  # Convert the start string to numbers (vectorize)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0     # Controls the randomness of predictions

  # Reset the model's state before generating the text
  # model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)

    predictions = tf.squeeze(predictions, 0)

    # Adjust predictions by temperature
    predictions = predictions / temperature

    # Sample the predicted character using categorical distribution
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # Pass the predicted character as the next input to the model
    input_eval = tf.expand_dims([predicted_id], 0)

    # Append the predicted character to the generated text
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [78]:
input = "ROMEO: "
print(generate_text(model, start_string=input))

ROMEO: What, is Ty
Glambess many gentlewoman's Mannar,
live no more know thou hadst:
If, then may not, Cambio!

QUEEN ELIZABETH:
What if you wear you?

Cape:
In hand of Coriolity I have heard,
So man sweet so thy love they die IJTAND:
Why, nay, I will tell thee!--due full extremis
'rd in quern other friends.

CAMILLO:
I cannot dren,
We will poor whether: I am dead!
Why then the duke shall not be delivered,
My most acquaint planters of thy brothers:
Remember what you shall proper vineant
To the door for shame was knowl'd;
Thou art's uhe of the Capito of death?
I'll hear the house may carry it with capers
Will we have known thee; Richard come to hers,
His kindred for his purpose to be hour.

BRAKENBURY:
Though time ask it to you. Calls
To capt it in the crowned cockling son should be worth to be so
